Solo Exploration and Presentation:

    1.Choose and download another data set from the UN data http://data.un.org/Explorer.aspx to merge with your data and explore. Prepare a short (< 5 minute) presentation of your findings. Report any interesting correlations you find. Include visualizations and consider adding interactivity with ipywidgets. This presentation can be done either in a Jupyter Notebook or using another presentation software, such as PowerPoint. (Check out Jupyter Slides if you have time. This allows you to turn your jupyter notebook into a slideshow.
    2.If time allows, check out the plotly library to add additional interactivity to your plots. https://plotly.com/python/plotly-express/)

Todo:
Can the country name updates to improve merging be done with a dictionary?
Is there a more interesting presentation of the data than line graphs?
Should we split out the Middle East into its own "continent"? Would Asia basically vanish if we did?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import plotly.express as px
%matplotlib inline
pd.options.display.max_rows = 999

## Import and clean up the crude oil reserves data set

In [2]:
crude_df = pd.read_csv('../data/un_crude_reserves.csv')[: -2]

In [3]:
crude_df.head(2)

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes
0,Albania,Crude petroleum - reserves,2017.0,"Metric tons, thousand",26000.0,1.0
1,Albania,Crude petroleum - reserves,2016.0,"Metric tons, thousand",26000.0,1.0


In [4]:
crude_df.tail(2)

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes
2597,"Yugoslavia, SFR (former)",Crude petroleum - reserves,1991.0,"Metric tons, thousand",32000.0,NaN
2598,"Yugoslavia, SFR (former)",Crude petroleum - reserves,1990.0,"Metric tons, thousand",32000.0,NaN


In [5]:
#Drop & Rename
crude_df = crude_df.drop(columns='Quantity Footnotes')
crude_df = crude_df.rename(columns = {'Country or Area' : 'Country', 'Quantity' : 'Reserve_Qty'})
crude_df.head(2)

,Country,Commodity - Transaction,Year,Unit,Reserve_Qty
0,Albania,Crude petroleum - reserves,2017.0,"Metric tons, thousand",26000.0
1,Albania,Crude petroleum - reserves,2016.0,"Metric tons, thousand",26000.0


In [6]:
#To Int
crude_df['Reserve_Qty'] = crude_df['Reserve_Qty'].astype(np.int64)
crude_df['Year'] = crude_df['Year'].astype(np.int64)
#df['column name'] = df['column name'].astype(np.int64)

crude_df.head(2)

,Country,Commodity - Transaction,Year,Unit,Reserve_Qty
0,Albania,Crude petroleum - reserves,2017,"Metric tons, thousand",26000
1,Albania,Crude petroleum - reserves,2016,"Metric tons, thousand",26000


In [7]:
#Any stray units?
crude_df.groupby('Unit').count()

,Country,Commodity - Transaction,Year,Reserve_Qty
Unit,,,,
"Metric tons, thousand",2599,2599,2599,2599


In [8]:
#Any stray descriptions?
crude_df.groupby('Commodity - Transaction').count()

,Country,Year,Unit,Reserve_Qty
Commodity - Transaction,,,,
Crude petroleum - reserves,2599,2599,2599,2599


## Import and clean up the gdp dataset, merge the continents dataset

In [9]:
gdp_df = pd.read_csv('../gdp_per_capita.csv')[: -2]

In [10]:
gdp_df.head(2)

,Country or Area,Year,Value,Value Footnotes
0,Afghanistan,2018,1734.723214,NaN
1,Afghanistan,2017,1758.465636,NaN


In [11]:
gdp_df.tail(2)

,Country or Area,Year,Value,Value Footnotes
6727,Zimbabwe,1991,2906.272849,NaN
6728,Zimbabwe,1990,2819.549467,NaN


In [12]:
#Drop, Rename, To Int
gdp_df = gdp_df.drop(columns='Value Footnotes')
gdp_df = gdp_df.rename(columns = {'Country or Area' : 'Country', 'Value' : 'GDP_Per_Capita'})
gdp_df['Year']=gdp_df['Year'].astype(np.int64)
gdp_df.head(2)

,Country,Year,GDP_Per_Capita
0,Afghanistan,2018,1734.723214
1,Afghanistan,2017,1758.465636


In [13]:
continents = pd.read_csv('../data/continents.csv')
continents.head(2)

,Continent,Country
0,Asia,Afghanistan
1,Europe,Albania


In [14]:
#Look at the unmatched countries to see if any are significant: they're not, an inner join is okay without further cleanup
check_countries = gdp_df.merge(continents, on = 'Country', how = 'outer', suffixes = ('_gdp', '_con'))

In [15]:
#What do the unmatched look like?
check_countries.tail(30)

,Country,Year,GDP_Per_Capita,Continent
6723,Zimbabwe,1995.0,2736.486436,Africa
6724,Zimbabwe,1994.0,2768.309953,Africa
6725,Zimbabwe,1993.0,2572.870395,Africa
6726,Zimbabwe,1992.0,2591.007534,Africa
6727,Zimbabwe,1991.0,2906.272849,Africa
6728,Zimbabwe,1990.0,2819.549467,Africa
6729,Andorra,NaN,NaN,Europe
6730,Bahamas,NaN,NaN,North America
6731,Burma (Myanmar),NaN,NaN,Asia
6732,Cuba,NaN,NaN,North America


In [16]:
#What countries do we have?
gdp_df.groupby('Country').count()

,Year,GDP_Per_Capita
Country,,
Afghanistan,17,17
Albania,29,29
Algeria,29,29
Angola,29,29
Antigua and Barbuda,29,29
Arab World,29,29
Argentina,29,29
Armenia,29,29
Aruba,28,28


In [17]:
gdp_df = gdp_df.merge(continents, on = 'Country', how = 'inner')
gdp_df.head(2)

,Country,Year,GDP_Per_Capita,Continent
0,Afghanistan,2018,1734.723214,Asia
1,Afghanistan,2017,1758.465636,Asia


In [18]:
#Check to see if our years correspond between the two datasets
gdp_df['Year'].agg({'min','max'})

max    2018
min    1990
Name: Year, dtype: int64

In [19]:
crude_df['Year'].agg({'min','max'})

max    2017
min    1990
Name: Year, dtype: int64

## Do a bunch of checking of the new dataset

In [20]:
check_crude_gdp = crude_df.merge(gdp_df[gdp_df['Year']<=2017], on=['Country','Year'], how='outer')

In [21]:
check_crude_gdp.head(2)

,Country,Commodity - Transaction,Year,Unit,Reserve_Qty,GDP_Per_Capita,Continent
0,Albania,Crude petroleum - reserves,2017,"Metric tons, thousand",26000.0,11796.719185,Europe
1,Albania,Crude petroleum - reserves,2016,"Metric tons, thousand",26000.0,11352.101422,Europe


In [22]:
#What does the data look like?
check_crude_gdp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5600 entries, 0 to 5599
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Country                  5600 non-null   object 
 1   Commodity - Transaction  2599 non-null   object 
 2   Year                     5600 non-null   int64  
 3   Unit                     2599 non-null   object 
 4   Reserve_Qty              2599 non-null   float64
 5   GDP_Per_Capita           5156 non-null   float64
 6   Continent                5156 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 350.0+ KB


In [23]:
# Rename, To Int again
check_crude_gdp = check_crude_gdp.rename(columns={'Commodity - Transaction':'Commodity'})
check_crude_gdp['Reserve_Qty'] = check_crude_gdp['Reserve_Qty'].fillna(0).astype(np.int64)
check_crude_gdp['GDP_Per_Capita'] = check_crude_gdp['GDP_Per_Capita'].fillna(0).astype(np.int64)

In [24]:
# Improve the matching
crude_df['Country'][crude_df['Country']=='Venezuela (Bolivar. Rep.)'] = 'Venezuela'
crude_df['Country'][crude_df['Country']=='USSR (former)'] = 'Russia'
crude_df['Country'][crude_df['Country']=='Russian Federation'] = 'Russia'
crude_df['Country'][crude_df['Country']=='Sudan (former)'] = 'Sudan'
crude_df['Country'][crude_df['Country']=='Iran (Islamic Rep. of)'] = 'Iran'

<ipython-input-24-cd31c85357b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crude_df['Country'][crude_df['Country']=='Venezuela (Bolivar. Rep.)'] = 'Venezuela'
<ipython-input-24-cd31c85357b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crude_df['Country'][crude_df['Country']=='USSR (former)'] = 'Russia'
<ipython-input-24-cd31c85357b2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crude_df['Country'][crude_df['Country']=='R

In [25]:
check_crude_gdp.head(2)

,Country,Commodity,Year,Unit,Reserve_Qty,GDP_Per_Capita,Continent
0,Albania,Crude petroleum - reserves,2017,"Metric tons, thousand",26000,11796,Europe
1,Albania,Crude petroleum - reserves,2016,"Metric tons, thousand",26000,11352,Europe


In [26]:
#Are all countries reporting for all years?
check_crude_gdp[['Country','Year']][check_crude_gdp['Commodity'].isnull()].groupby('Country').count()

,Year
Country,
Afghanistan,16
Antigua and Barbuda,28
Armenia,28
Aruba,28
Azerbaijan,2
Bangladesh,1
Belarus,7
Belgium,28
Belize,17


In [27]:
#Merge the data
crude_gdp = crude_df.merge(gdp_df[gdp_df['Year']<=2017], on=['Country','Year'], how='inner')

In [28]:
crude_gdp.head(2)

,Country,Commodity - Transaction,Year,Unit,Reserve_Qty,GDP_Per_Capita,Continent
0,Albania,Crude petroleum - reserves,2017,"Metric tons, thousand",26000,11796.719185,Europe
1,Albania,Crude petroleum - reserves,2016,"Metric tons, thousand",26000,11352.101422,Europe


In [29]:
crude_gdp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2261 entries, 0 to 2260
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Country                  2261 non-null   object 
 1   Commodity - Transaction  2261 non-null   object 
 2   Year                     2261 non-null   int64  
 3   Unit                     2261 non-null   object 
 4   Reserve_Qty              2261 non-null   int64  
 5   GDP_Per_Capita           2261 non-null   float64
 6   Continent                2261 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 141.3+ KB


In [30]:
crude_gdp['Reserve_Qty'] = crude_gdp['Reserve_Qty'].fillna(0).astype(np.int64)
crude_gdp['GDP_Per_Capita'] = crude_gdp['GDP_Per_Capita'].fillna(0).astype(np.int64)

In [31]:
crude_gdp.head(2)

,Country,Commodity - Transaction,Year,Unit,Reserve_Qty,GDP_Per_Capita,Continent
0,Albania,Crude petroleum - reserves,2017,"Metric tons, thousand",26000,11796,Europe
1,Albania,Crude petroleum - reserves,2016,"Metric tons, thousand",26000,11352,Europe


In [32]:
crude_gdp.tail(2)

,Country,Commodity - Transaction,Year,Unit,Reserve_Qty,GDP_Per_Capita,Continent
2259,Yemen,Crude petroleum - reserves,1992,"Metric tons, thousand",544000,3561,Asia
2260,Yemen,Crude petroleum - reserves,1991,"Metric tons, thousand",544000,3466,Asia


In [33]:
#Slice out the columns we want
countries_all_years = crude_gdp[['Year','Continent','Country','Reserve_Qty']]

In [34]:
#Divide reserves by 1000
countries_all_years['Reserve_Qty_000'] = countries_all_years['Reserve_Qty'] / 1000

<ipython-input-34-e772ec80249d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countries_all_years['Reserve_Qty_000'] = countries_all_years['Reserve_Qty'] / 1000


In [35]:
#Make a new dataframe of the sum of reserves by continent and year
plot_sum = countries_all_years.groupby(['Continent','Year'])['Reserve_Qty_000'].sum().to_frame().reset_index()

In [88]:
#Show changes in reserves by year for each continent
df1=plot_sum.pivot(index='Year',columns='Continent',values='Reserve_Qty_000')
px.line(df1, 
        title='Petroleum Reserves By Continent - Million Metric Tons',
       labels={'value':'Reserves'})

In [37]:
#slice out the countries who have reported reserves in 1990, only want countries reporting from the beginning
slice_countries_1990 = crude_gdp[(crude_gdp['Year']==1990) & (crude_gdp['Reserve_Qty']>0)].sort_values('Reserve_Qty', ascending=False)

In [80]:
slice_countries_1990['Country'].count()

59

In [39]:
#slice out the countries in the top 10 for 2017
slice_countries_2017 = crude_gdp[(crude_gdp['Year']==2017) 
          & (crude_gdp['Country'].isin(slice_countries_1990['Country']))].sort_values('Reserve_Qty', ascending=False).head(10)

In [84]:
#slice_countries_2017

In [41]:
#Filter the original dataset down to the top 20
top_countries_all_years = crude_gdp[['Country', 'Year', 'Reserve_Qty']][crude_gdp['Country'].isin(slice_countries_2017['Country'])]

In [42]:
top_countries_all_years['Reserve_Qty_000'] = top_countries_all_years['Reserve_Qty'] / 1000

In [43]:
top_countries_all_years.head(2)

,Country,Year,Reserve_Qty,Reserve_Qty_000
418,Canada,2017,27754600,27754.6
419,Canada,2016,27754600,27754.6


In [44]:
top_countries_all_years = top_countries_all_years.drop(columns = 'Reserve_Qty')

In [87]:
df2=top_countries_all_years.pivot(index='Year',columns='Country',values='Reserve_Qty_000')
px.line(df, 
        title='Petroleum Reserves Top Ten Countries - Million Metric Tons',
       labels={'value':'Reserves'})

In [81]:
#gdp_df.groupby('Country').count()

In [82]:
#crude_df.groupby('Country')['Reserve_Qty'].sum()

In [83]:
#crude_df.merge(gdp_df, how='left', on = 'Country').groupby('Country').count()

In [59]:
#gdp_df[gdp_df['Country'].str.contains('Syria')]

DataCamp scatter plot example

Scatter plot
plt.scatter(x = gdp_cap, y = life_exp, s = np.array(pop) * 2, c = col, alpha = 0.8)

Previous customizations
plt.xscale('log') plt.xlabel('GDP per Capita [in USD]') plt.ylabel('Life Expectancy [in years]') plt.title('World Development in 2007') plt.xticks([1000,10000,100000], ['1k','10k','100k'])

Additional customizations
plt.text(1550, 71, 'India') plt.text(5700, 80, 'China')

Add grid() call
plt.grid(True)

Show the plot
plt.show()